# Webscraping Hotel Chocolat's Reviews on Trustpilot using BeautifulSoup4

In [1]:
#importing libraries
import csv
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import pandas as pd
import numpy as np
import time

### Request Headers

In [3]:
#get relevant user agent
#https://httpbin.org/anything
#https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending
my_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.co.uk/",
    "Sec-Ch-Ua": "\"Chromium\";v=\"110\", \"Not A(Brand\";v=\"24\", \"Google Chrome\";v=\"110\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"", 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "cross-site", 
    "Sec-Fetch-User": "?1", 
    "Upgrade-Insecure-Requests": "1",}

In [4]:
#check where traffic is coming from
#without setting the my_headers above, it will typically return python
#https://requests.readthedocs.io/en/latest/user/advanced/
session = requests.Session()
session.headers = my_headers

In [5]:
%%time
session.headers['User-Agent']

Wall time: 0 ns


'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'

In [6]:
session.headers['Accept-Language']

'en-US,en;q=0.9'

### Website HTML

In [7]:
# URL for scraping data
hotel_choc = 'https://uk.trustpilot.com/review/hotelchocolat.com'

In [8]:
#response
chocolat = session.get(hotel_choc, headers = my_headers)
chocolat_soup = BeautifulSoup(chocolat.text, 'html.parser')
print(chocolat_soup.prettify())

<!DOCTYPE html>
<html lang="en-GB">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="Trustpilot" name="application-name"/>
  <meta content="#1c1c1c" name="theme-color"/>
  <link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link color="#00b67a" href="https://cdn.trustpilot.net/brand-assets/1.9.0/favicons/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="Trustpilot" name="apple-mobile-web-app-title"/

### Show how data for one page will look like when scraped

In [9]:
%%time

#extract from single page
def choco_single(url):
    response = session.get(url, headers = my_headers)
    choco_soup = BeautifulSoup(response.text, 'html.parser')
    
    div_a = choco_soup.find_all("div", {"class": "styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ"})
    chocolat_data = []
    
    for div in div_a:
        name  = div.find("span", {"class": "typography_heading-xxs__QKBS8 typography_appearance-default__AAY17"})
        no_rev = div.find("span", {"class": "typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l"})
        rating = div.find("div", {"class": "styles_reviewHeader__iU9Px"})
        rev_title = div.find("h2", {"class": "typography_heading-s__f7029 typography_appearance-default__AAY17"})
        rev = div.find("p", {"class": "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn"})
        rev_date = div.find("p", {"class": "typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_color-black__5LYEn"})
        rev_reply = div.find("p", {"class": "typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX"})
        
        name_text = name.text.strip() if name else None
        no_rev_text = no_rev.text.strip() if no_rev else None
        rating_text = rating.get("data-service-review-rating")
        rev_title_text = rev_title.text.strip() if rev_title else None
        rev_text = rev.text.strip() if rev else None
        
        rev_date_text = rev_date.text.split(":")[1].strip() if rev_date else None
        
        
        rev_reply_text = rev_reply.text.strip() if rev_reply else None
        
        chocolat_data.append({"Name": name_text,
                              "N_Reviews": no_rev_text,
                              "Ratingss": rating_text,
                              "Title": rev_title_text,
                              "Review": rev_text,
                              "Date": rev_date_text,
                              "Reply": rev_reply_text
            
        })
        
    return chocolat_data

Wall time: 0 ns


In [10]:
%%time
url_2 = 'https://uk.trustpilot.com/review/hotelchocolat.com'
chocolat_data = choco_single(url_2)
chocolat_data

Wall time: 78.9 ms


[{'Name': 'G.M.',
  'N_Reviews': '7 reviews',
  'Ratingss': '3',
  'Title': 'Hot chocolate isn’t hot enough',
  'Review': 'Quality product which was well packaged. Unfortunately the hot chocolate isn’t hot enough. The first batch was actually cold, which wasted a sachet from the selection box. The second batch was slightly better but still not great. I’ll give it a few more tries and if it doesn’t improve, it will have to be returned. Disappointed. My young daughter was very excited about it.',
  'Date': '24 February 2023',
  'Reply': None},
 {'Name': 'Mrs Carol Winmill',
  'N_Reviews': '27 reviews',
  'Ratingss': '1',
  'Title': 'I have a subscription for cafe latte…',
  'Review': 'I have a subscription for cafe latte £13.50 a box but you’ve taken double that this month numerous emails and I’m still getting run around not great customer service. I had a £5 voucher but I’m not allowed to use it with my subscription and if I use it for chocolate I’m charged £4.95 delivery. What’s the po

In [11]:
len (chocolat_data)

20

### Getting the data across multiple pages

In [13]:
%%time
#scraping from 200 pages
choco_data = []
for i in range(1, 201):
    choco_url = 'https://uk.trustpilot.com/review/hotelchocolat.com?page={}'.format(i)
    choco_data += choco_single(choco_url)
    if i % 10 == 0:
        time.sleep(150)
        
print (choco_data)

[{'Name': 'G.M.', 'N_Reviews': '7 reviews', 'Ratingss': '3', 'Title': 'Hot chocolate isn’t hot enough', 'Review': 'Quality product which was well packaged. Unfortunately the hot chocolate isn’t hot enough. The first batch was actually cold, which wasted a sachet from the selection box. The second batch was slightly better but still not great. I’ll give it a few more tries and if it doesn’t improve, it will have to be returned. Disappointed. My young daughter was very excited about it.', 'Date': '24 February 2023', 'Reply': None}, {'Name': 'Mrs Carol Winmill', 'N_Reviews': '27 reviews', 'Ratingss': '1', 'Title': 'I have a subscription for cafe latte…', 'Review': 'I have a subscription for cafe latte £13.50 a box but you’ve taken double that this month numerous emails and I’m still getting run around not great customer service. I had a £5 voucher but I’m not allowed to use it with my subscription and if I use it for chocolate I’m charged £4.95 delivery. What’s the point of the voucher.',

In [14]:
len(choco_data)

4000

In [16]:
choco_df = pd.DataFrame(choco_data)
choco_df.head(10)

,Name,N_Reviews,Ratingss,Title,Review,Date,Reply
0,G.M.,7 reviews,3,Hot chocolate isn’t hot enough,Quality product which was well packaged. Unfor...,24 February 2023,None
1,Mrs Carol Winmill,27 reviews,1,I have a subscription for cafe latte…,I have a subscription for cafe latte £13.50 a ...,23 February 2023,None
2,MRS DOMINIQUE HAZZARD,6 reviews,5,VELVETISER AND SUB Just bought the velvetiser ...,Just bought the velvetiser and a hot chocolate...,23 February 2023,None
3,Anonymous,2 reviews,4,Mixed bag,The rose and violet creams are beyond compare....,23 February 2023,"Hi there, thanks for taking the time to leave ..."
4,Gail Pollard,17 reviews,4,Always delivered on time,I purchased a Velvetiser April 2022 and have a...,24 February 2023,None
5,Jackie,5 reviews,5,A gem brand!,Cannot go wrong with Hotel Chocolate! Consiste...,23 February 2023,"We aim to please, Jackie! Thank you so much fo..."
6,Mr Abbott,5 reviews,5,Excellent customer service,After using my initial pack of chocolat sachet...,22 February 2023,We are thrilled that you love your Velvetiser ...
7,L McArdle,2 reviews,5,Fantastic.,Great communication during delivery wait. Fab ...,24 February 2023,None
8,Jenniep,5 reviews,5,Wish I had brought it sooner to enjoy,I had been thinking about buying a Velvetiser ...,11 February 2023,"This is what we like to hear Jennie, we hope t..."
9,liz Ch. Ferro,1 review,1,expired products,I bought some products on offer. i never imagi...,19 February 2023,Hi Liz. We are so sorry to hear about your rec...


In [17]:
choco_df.to_excel('20230226_Rchoco_data-v1.xlsx')